### Imports:

In [ ]:
from gensim.models import KeyedVectors
from gensim.models import FastText

import pandas as pd
from sklearn.model_selection import train_test_split

from random import shuffle
from pickle import dump, load
from numpy import array
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Embedding, LSTM
from keras.optimizers import SGD


### Model:

In [ ]:
embedding_model = KeyedVectors.load_word2vec_format(r'embeddings\Word2Vec\cbow_s100.txt')

In [ ]:
data = pd.read_csv('train.csv')
x_train, x_test, y_train, y_test = train_test_split(data['samples'], data['Intent'], test_size = 0.3, shuffle = True, stratify = data['Intents'], random_state = 7)

# le = LabelEncoder()
# le.fit(y_train)
# y_train = le.transform(y_train)
# y_test = le.transform(y_test)

# tokenizer_ = Tokenizer()
# tokenizer_.fit_on_texts(X_train)
# print(f"Train Document Count: \n{tokenizer_.document_count}\n")

# def convert_to_padded(tokenizer, docs):
#     embedded = tokenizer.texts_to_sequences(docs)
#     padded = pad_sequences(embedded, maxlen = max_length, padding = 'post')
#     return padded

# vocab_size = len(tokenizer_.word_counts) + 1
# print(f'Vocab size:\n{vocab_size}')

# padded_X_train = convert_to_padded(tokenizer = tokenizer_, docs = X_train)
# padded_X_test = convert_to_padded(tokenizer = tokenizer_, docs = X_test)
# print(f'padded_X_train\n{padded_X_train}')
# print(f'padded_X_val\n{padded_X_test}')

# max_length = len(max(padded_X_train, key = len))
# print(f'Max length:\n{max_length}')

# embeddings_index = {}
# f = open('models/glove.twitter.27B/glove.twitter.27B.50d.txt')
# for line in f:
#     values = line.split()
#     word = values[0]
#     coefs = np.asarray(values[1:], dtype='float32')
#     embeddings_index[word] = coefs
# f.close()
# print('Found %s word vectors.' % len(embeddings_index))

# word_index = tokenizer_.word_index
# EMBEDDING_DIM = 50
# embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
# for word, i in word_index.items():
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None:
#         embedding_matrix[i] = embedding_vector

In [ ]:
def make_model(qtd_labels,max_length):
    model = Sequential()

    model.add(Embedding(vocab_size, embedding_matrix.shape[1], input_length = max_length, trainable = False, weights = [embedding_matrix]))

    model.add(Bidirectional(LSTM(128, return_sequences=True, recurrent_dropout=0.1, dropout=0.1), 'concat'))
    model.add(Dropout(0.5))

    model.add(LSTM(128, return_sequences=False, recurrent_dropout=0.1, dropout=0.1))
    model.add(Dropout(0.5))

    model.add(Dense(128, input_shape=(max_length), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(qtd_labels, activation='softmax'))

def train_model(df):
    